# testing the modules

In [1]:
from pyapp.observation.phoneix import traced_agent,ph_instrumentor

# -------------------model -------------------#
from pyapp.model_connection.lm.langchain.litellm import get_model_mlflow_llamacpp, ModelConfig
### --- config --- ###
from pathlib import Path
from dotenv import load_dotenv
import os 
here = Path("./").resolve()
print(Path(here / "config.env").exists())
load_dotenv(here / "config.env")

mlflow_port = os.getenv("MLFLOW_PORT") or "5001"
tracking_uri=f"http://localhost:{mlflow_port}"
config_path=here / "model_dir"

### --- model --- ###
model_config = ModelConfig(model_name="rag_model",port=8080,gguf_relative_path="model_path/artifacts/model.gguf")
model = get_model_mlflow_llamacpp(tracking_uri,config_path,model_config , stream=True , mock_response="Hello world", temperature=0, max_tokens=20)



May 8, 2025 > 13:29:58 |  pyapp.observation.phoneix : 17 |   ℹ️ INFO  |  Starting Phoenix observation  
May 8, 2025 > 13:29:58 |  taskpy.main : 28 |   🐞 DEBUG  |  ['task', '--dir', PosixPath('/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/app/pyapp/src/pyapp/observation'), 'status']  
May 8, 2025 > 13:29:58 |  taskpy.main : 32 |   ❌ ERROR  |  task: [status] docker ps | grep lmorbits-obeservation | wc -l
  
May 8, 2025 > 13:29:58 |  taskpy.main : 34 |   ℹ️ INFO  |         1
  
May 8, 2025 > 13:29:58 |  pyapp.observation.instance : 34 |   ℹ️ INFO  |  Phoenix is already running  
True
May 8, 2025 > 13:29:59 |  pyapp.serve_integration.mlflow_llamacpp : 23 |   ℹ️ INFO  |  Config path: /Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/app/pyapp/notebooks/model_dir  
May 8, 2025 > 13:29:59 |  pyapp.serve_integration.mlflow_llamacpp : 24 |   ℹ️ INFO  |  MLflow client: <mlflow.tracking.client.MlflowClient object at 0x117aefc50>  
Ma

In [2]:
from pyapp.observation.phoneix import async_generator_traced_agent

In [5]:
# -------------------traced_agent -------------------#
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{input}"),
])

chain = prompt | model

@traced_agent(name="app1")
def assistant(messages: list[dict], session_id: str):
    # now you only have to do your business logic
   return chain.invoke(messages) 

@traced_agent(name="app2")
def assistant2(messages: list[dict], session_id: str):
    # now you only have to do your business logic
    return chain.invoke(messages)


@async_generator_traced_agent(name="app3")
async def assistant3(messages: list[dict], session_id: str):
    answer, trace_url =  assistant(messages, session_id)
    answer2, trace_url2 = assistant2(messages, session_id)
    answer = ""
    async for chunks in chain.astream(messages):
        answer += chunks.content
        yield chunks


messages = [
  {"role": "user", "content": "hi! im bob"}
]
import uuid
session_id = str(uuid.uuid4())
# answer, trace_url = await assistant3(messages, session_id)
# print(trace_url)
linked_url = None
async for chunk in assistant3(messages, session_id):
    print(chunk[0].content)
    linked_url = chunk[1]
print(linked_url)

I0000 00:00:1746700531.673991  428614 chttp2_transport.cc:1201] ipv6:%5B::1%5D:4317: Got goaway [11] err=UNAVAILABLE:GOAWAY received; Error code: 11; Debug Text: ping_timeout {grpc_status:14, http2_error:11, created_time:"2025-05-08T13:35:31.673972+03:00"}


[
{'
role
':
 '
user
',
 '
content
':
 '
hi
!
 im
 bob
'}
]




```

http://localhost:6006/projects/UHJvamVjdDoy/traces/5ca06be7d46098e702522b670f896581?selectedSpanNodeId=U3Bhbjoy
